In [1]:
from lib import TTConfig, TTOption, TTOptionSide
from tastytrade import DXLinkStreamer, Session, Account
from tastytrade.instruments import Option, Equity
from tastytrade.dxfeed import EventType
import asyncio

In [2]:
config = TTConfig()

In [3]:
session = Session(config.username, config.password, is_test=not config.use_prod)

In [4]:
Account.get_accounts(session)

[Account(account_number='5WY75488', opened_at=datetime.datetime(2024, 10, 1, 17, 32, 11, 360000, tzinfo=TzInfo(UTC)), nickname='Individual', account_type_name='Individual', is_closed=False, day_trader_status=False, is_firm_error=False, is_firm_proprietary=False, is_futures_approved=False, is_test_drive=False, margin_or_cash='Margin', is_foreign=True, created_at=datetime.datetime(2024, 10, 1, 17, 32, 11, 372000, tzinfo=TzInfo(UTC)), external_id='A20360ae9-1802-48a2-8f04-f21d81101e1e', closed_at=None, funding_date=None, investment_objective='SPECULATION', liquidity_needs='SOMEWHAT_IMPORTANT', risk_tolerance=None, investment_time_horizon=None, futures_account_purpose=None, external_fdid=None, suitable_options_level='No Restrictions', submitting_user_id=None)]

In [5]:
session

In [6]:
option_for_request = TTOption('SPXW', '241014', TTOptionSide.CALL, 5850)

In [7]:
option_served = Option.get_option(session, option_for_request.symbol)

In [8]:
option_served.streamer_symbol

'.SPXW241014C5850'

In [9]:
num_events = 10
gotten_so_far = 0
async with DXLinkStreamer(session) as streamer:
    await streamer.subscribe(EventType.QUOTE, [option_served.streamer_symbol])
    quotes = []
    try:
        async for quote in streamer.listen(EventType.QUOTE):
            gotten_so_far += 1
            print(f'Got quote no. {gotten_so_far}, ask price: {quote.bidPrice}, bid price: {quote.askPrice} ')
            quotes.append(quote)
            if len(quotes) >= num_events:
                break
    except asyncio.CancelledError:
        await streamer.cancel_channel(EventType.QUOTE)
        await streamer.close()
        print('Unsubscribed')


Got quote no. 1, ask price: 13.9, bid price: 14.2 
Got quote no. 2, ask price: 13.8, bid price: 14.1 
Got quote no. 3, ask price: 13.8, bid price: 14.0 
Got quote no. 4, ask price: 14.1, bid price: 14.3 
Got quote no. 5, ask price: 14.4, bid price: 14.7 
Got quote no. 6, ask price: 14.5, bid price: 14.7 
Got quote no. 7, ask price: 14.9, bid price: 15.2 
Got quote no. 8, ask price: 14.8, bid price: 15.1 
Got quote no. 9, ask price: 14.9, bid price: 15.2 
Got quote no. 10, ask price: 14.7, bid price: 14.9 


In [14]:
streamer._wss_url

'wss://tasty-live-ws.dxfeed.com/realtime'

In [16]:
streamer = await DXLinkStreamer.create(session)

In [18]:
quote = await streamer.get_event(EventType.QUOTE)
print(quote)

eventSymbol='.SPXW241010C5750' eventTime=0 sequence=0 timeNanoPart=0 bidTime=0 bidExchangeCode='C' askTime=0 askExchangeCode='C' bidPrice=Decimal('43.9') askPrice=Decimal('47.0') bidSize=Decimal('101.0') askSize=Decimal('101.0')


In [ ]:
await streamer.cancel_channel(EventType.QUOTE)
await streamer.close()

In [17]:
session.destroy()